In [11]:
import time
import random
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import clear_output

<h1 style="font-size:40px;"> <center> CONSTANTS </center> </h1>

In [47]:
KEYWORD = "cinayet"
START_DATE = "01/01/2022"
FINISH_DATE = "31/12/2022"
MAIN_CATEGORY = "gundem"    

<h1 style="font-size:40px;"> <center> COLLECT URLS </center> </h1>

In [48]:
start = time.time()

urls = []
titles = []
abstracts = []
dates = []

service = Service('/chromedriver.exe')
service.start()
driver = webdriver.Remote(service.service_url)
pages = list(range(1,180))
last_url_count = 0

for page in pages:
    try:
        search_url = f"https://www.hurriyet.com.tr/arama/#/?page={page}&key={KEYWORD}&order=Yeniden%20Eskiyewhere=hurriyet&how=Article&and={KEYWORD}&startDate={START_DATE}&finishDate={FINISH_DATE}&platform=hurriyet&mainCategory=/{MAIN_CATEGORY}/&isDetail=true"
        driver.get(search_url)
        time.sleep(3)
        div_content_tags = driver.find_elements(By.XPATH,"//div[@class='hs-cnn-content']")
        for div_content_tag in div_content_tags:
            a_tag = div_content_tag.find_element(By.TAG_NAME,"a")
            urls.append(a_tag.get_attribute("href"))
        p_title_tags = driver.find_elements(By.XPATH,"//p[@class='hs-cnnc-title']")
        for p_title_tag in p_title_tags:
            titles.append(p_title_tag.get_attribute("innerHTML"))
        p_text_tags = driver.find_elements(By.XPATH,"//p[@class='hs-cnnc-text']")
        for p_text_tag in p_text_tags:
            abstracts.append(p_text_tag.get_attribute("innerHTML"))
        p_date_tags = driver.find_elements(By.XPATH,"//p[@class='hs-cnncc-date']")
        for p_date_tag in p_date_tags:
            dates.append(p_date_tag.get_attribute("innerHTML")) 
        if last_url_count == len(urls):
            break
        last_url_count = len(urls)
        clear_output(wait=True)
        print(f"{len(urls)} urls were fetched so far (page number-{page})", sep=' ', end='', flush=True)
    except Exception as e: 
        print(e)
        print("Last Page")
        break
    
driver.quit()  
print(f"\n\n{len(urls)} urls were collected in {time.time() - start} second.")
print(f"{round(len(urls) / (time.time() - start), 2)} urls were collected per second on average.")

1592 urls were fetched so far (page number-160)

1592 urls were collected in 650.4402616024017 second.
2.45 urls were collected per second on average.


In [49]:
scraped_urls = pd.DataFrame({"url":urls, "title":titles, "abstract":abstracts, "date":dates })
#scraped_urls.set_index("date")
scraped_urls

,url,title,abstract,date
0,http://www.hurriyet.com.tr/gundem/ankarada-sil...,"Ankara’da silahlı kavga: 3 ölü, 1 yaralı",Ankara’nın Mamak ilçesinde bir kafede çıkan si...,31.12.2022
1,http://www.hurriyet.com.tr/gundem/copu-dokmeye...,Çöpü dökmeyen kardeşini 38 yerinden bıçaklayar...,Ankara'da çöpü dökmesini isteyince kendisini t...,30.12.2022
2,http://www.hurriyet.com.tr/gundem/bayrampasada...,Bayrampaşa'da korkunç olay! Önce darbettiler s...,İstanbul Bayrampaşa'da bir eğlence mekanında M...,30.12.2022
3,http://www.hurriyet.com.tr/gundem/silanin-kati...,Sıla’nın katiline ağırlaştırılmış müebbet,16 yaşındaki Sıla Şentürk’ü boğazını keserek ö...,30.12.2022
4,http://www.hurriyet.com.tr/gundem/cem-garipogl...,Cem Garipoğlu'nun mezarının açılmasına verilen...,13 yıl önce vahşice öldürülen Münevver Karabul...,29.12.2022
...,...,...,...,...
1587,http://www.hurriyet.com.tr/gundem/kirsehirde-a...,"Kırşehir'de ağabey-kardeş, 23 gün arayla öldür...",Kırşehir'in Kaman ilçesinde husumetlileri tara...,2.1.2022
1588,http://www.hurriyet.com.tr/gundem/eglence-cina...,Eğlence cinayetine ömür boyu hapis cezası,"Bursa’da, arkadaşlarıyla eğlenmek için gittiği...",1.1.2022
1589,http://www.hurriyet.com.tr/gundem/bagcilarda-y...,"Bağcılar’daki cinayet, intihar çıktı: Hepinizi...",Bağcılar’da yılbaşı gecesi 14 yaşındaki Hüseyi...,1.1.2022
1590,http://www.hurriyet.com.tr/gundem/arkadas-grub...,Arkadaş grubunun yılbaşı eğlencesinde bıçaklı ...,"Tokat'ın Erbaa ilçesinde Nurettin Öztürk (56),...",1.1.2022


In [50]:
file_name = KEYWORD + "-" + START_DATE[-4:] + "-" + MAIN_CATEGORY
scraped_urls.to_excel(f"outputs/scraped_urls-{file_name}.xlsx")
scraped_urls.to_csv(f"outputs/scraped_urls-{file_name}.csv")

<h1 style="font-size:40px;"> <center> COLLECT NEWS CONTENTS </center> </h1>

In [ ]:
start = time.time()

with open('news-urls.txt') as f:
    news_urls = f.readlines()
    
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"
}
    
url_texts = [] 
f = open("scraped_news_contents.txt", "a")

for url in news_urls:
    r = requests.get(url.strip(), headers=headers)
    print(r)
    soup = BeautifulSoup(response.content, "html.parser")


<h1 style="font-size:40px;"> <center> TEXT ANALYZING </center> </h1>

In [ ]:
import openai

In [ ]:
def generate_response(prompt: str = "I have no question", creativity: float = 0):
    """
    Args:
        prompt (str): user input prompt

    Returns:
        str: response of chatgpt
    """
    completions = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=creativity / 10,
    )
    return completions.choices[0].text

In [ ]:
with open('key.txt') as f:
    openai.api_key = f.readlines()[0]

In [ ]:
start = time.time()

with open('scraped_news_contents.txt') as f:
    scraped_news_content = f.read()
f = open("analyzed_news_details.txt", "a")
i=0
for news_content in scraped_news_content.split("DELIMITER"):
    print(i)
    i+=1
    question = "Please tell me the name and gender of the person who kills, \
    the name and the gender of the person who is killed, and date in a uniform format like this: name_of_killer, gender_of killer, name_of_killed, gender_of_killed, date   \
    according to this new: (Thank you) "
    question += news_content
    answer = generate_response(question)
    print("answer given")
    result = answer + "\nDELIMITER\n"
    f.write(result)
    print(answer)
    
f.close()
print(f"{len(urls)} urls were fetched in {time.time() - start} second.")
print(f"{round(len(urls) / (time.time() - start), 2)} urls were fetched per second on average.")